### Social Network Analysis
##### Combine new variables and create diversity indices
##### Aug 30, 2021

In [2]:
import os
import pandas as pd
import numpy as np
from math import log as ln
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt

In [3]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', None)

In [4]:
df = pd.read_sas('vr_sntwk_2008_m_0641s_v2_16.sas7bdat',format = 'sas7bdat', encoding="unicode_escape")
demo_df = pd.read_sas('vr_wkthru_ex09_1_1001s_16.sas7bdat',format = 'sas7bdat', encoding="unicode_escape")
apoe_df = pd.read_sas('coh_off_apoe_16.sas7bdat',format = 'sas7bdat', encoding="unicode_escape")
ex_df = pd.read_sas('ex1_7s_v2_16.sas7bdat',format = 'sas7bdat', encoding="unicode_escape",index='idr')
ed_df = pd.read_sas('vr_np_2018_a_1185s_19.sas7bdat',format = 'sas7bdat', encoding="unicode_escape")


/usr/local/lib/python3.9/site-packages/pandas/io/sas/sas7bdat.py:800: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  rslt[name] = self._byte_chunk[jb, :].view(dtype=self.byte_order + "d")
/usr/local/lib/python3.9/site-packages/pandas/io/sas/sas7bdat.py:809: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  rslt[name] = self._string_chunk[js, :]


In [5]:
ed_df['idr'] = ed_df['idr'].astype(int).astype(str)
ed_df = ed_df.groupby('idr',as_index=False).first()
ed_df['EDU_COLLEGE'] = ed_df['EDUCG'].apply(lambda x: 'Some college' if x>1 else ('No college' if x<=1 else x))
ed_df = ed_df[['idr','EDUCG','EDU_COLLEGE']]

In [6]:
ex_df = ex_df.iloc[:,446:509].reset_index()
ex_df['idr'] = ex_df['idr'].astype(int).astype(str)

In [7]:
apoe_df['idr'] = apoe_df['idr'].astype(int).astype(str)

In [8]:
age_cols = [x for x in demo_df.columns if "AGE" in x]
demo_df = demo_df[['idr','SEX']+age_cols]
demo_df['idr'] = demo_df['idr'].astype(int).astype(str)

In [9]:
w1 = df[(df['idtype']==1) & 
    ((df['alteridtype']==0)&(df['SPELLBEGIN']<29) | (df['alteridtype']==1) & (df['SPELLBEGIN']<44)) &
    (df['SPELLEND']>44)].reset_index(drop=True)
w1[['idr','sharealterid']] = w1[['idr','sharealterid']].astype(int).astype(str)

In [10]:
w1['RELTYPE'] = w1['ALTERTYPE'].apply(lambda x: 'NONRELATIVE' if 'NR' in x else 'RELATIVE' )
w1_df = w1.groupby(['idr','sharealterid','RELTYPE'],as_index=False).first()

In [11]:
w1_df.idr.nunique()

4941

In [12]:
apoe_df['APOE4'] = apoe_df['APOE'].apply(lambda x: '+' if x in [34,24, 44] else ('-' if x not in [34,24,44] else x))

In [13]:
apoe_df['APOE'].value_counts()
#apoe4 risk of dementia
#34, 24, or 44

33.0    3464
34.0    1038
23.0     672
24.0      94
44.0      89
22.0      24
Name: APOE, dtype: int64

In [14]:
apoe_df.isna().sum()

idr       0
idtype    0
APOE      0
APOE4     0
dtype: int64

In [15]:
apoe_df['APOE4'].value_counts()

-    4160
+    1221
Name: APOE4, dtype: int64

#### Lots of duplicates, probably from ego/alter ties fulfilling the criteria for multiple categories. Drop duplicates, only keeping first entry for now
#### TO DO: should keep closest tie category 

In [16]:
comb_df = w1_df.merge(demo_df,on='idr',how='left').merge(ex_df,on='idr',how='left').merge(ed_df,on='idr',how='left').merge(apoe_df.drop('idtype',axis=1),on='idr',how='left')

In [17]:
comb_df['AGE35']= comb_df['AGE1'].apply(lambda x: '<35' if x<35 else '>=35')
comb_df['AGE65']= comb_df['AGE1'].apply(lambda x: '<65' if x<65 else '>=65')
comb_df['AGE_Q']= pd.cut(comb_df['AGE1'],4)
comb_df['AGE_10']= pd.cut(comb_df['AGE1'],range(0,110,10),labels=['<=10','<=20','<=30','<=40','<=50',
                                                                 '<=60','<=70','<=80','<=90','<=100'])

CES-D: G587-G606

Loneliness : G600

SNI: G645-G657

In [18]:
comb_df['CESD_TOT'] = comb_df.loc[:,'G587':'G606'].apply(lambda x: sum(x),axis=1)
comb_df['CESD_16'] = comb_df['CESD_TOT'].apply(lambda x: '>=16' if x>=16 else ('<16' if x<16 else x))
comb_df['CESD_20'] = comb_df['CESD_TOT'].apply(lambda x: '>=20' if x>=20 else ('<20' if x<20 else x))

In [19]:
comb_df['CESD_LONELINESS'] = comb_df['G600']
comb_df['LONELY_3'] = comb_df['CESD_LONELINESS'].apply(lambda x: '>=3' if x>=2 else ('<3' if x<3 else x))

In [20]:
comb_df['SNI_TOT'] = comb_df.loc[:,'G645':'G657'].apply(lambda x: sum(x),axis=1)
comb_df['SNI_2'] = comb_df['SNI_TOT'].apply(lambda x: '>2' if x>2 else ('<=2' if x<=2 else x))

In [21]:
# [(c,comb_df[c].isna().sum()) for c in comb_df.columns if comb_df[c].isna().sum()>0]

### Create Diversity Index

In [25]:
#individual network indices
#TO DO : get rid of for loop

def create_diversity_index(df,attr):
    shannon_list = []
    simpson_list = []
    ego_list = df['idr'].unique()
    
    for e in ego_list:
        #select network
        ego_df = df[df['idr']==e]
        alter_list = ego_df['sharealterid'].unique()
        #get network info using alters 'ego' attributes
        net_df = df[df['idr'].isin(alter_list)].groupby(['idr'],as_index=False).first()
        div_dict = dict(net_df[attr].value_counts())

        #NA if no tie info
        if div_dict:
        
        #shannon index
            shannon_h = 0
            for cat in div_dict:
                p = float(div_dict[cat]/sum(div_dict.values()))
                temp = float(p*(np.log(p)))
                if np.isfinite(temp):
                    shannon_h += temp      
            if shannon_h < 0:
                shannon_h = -(shannon_h)
            shannon_list.append(shannon_h)
        
        
        #simpson index
            simpson_d = 0
            for cat in div_dict:
                p = float(div_dict[cat]/sum(div_dict.values()))
                temp = float(p**2)
                simpson_d+= temp
            simpson_d = 1 - simpson_d
            simpson_list.append(simpson_d)

        
        else:
            shannon_list.append(np.nan)
            simpson_list.append(np.nan)

    new_df = pd.DataFrame({'idr':ego_list,(attr+'_SHANNON'):shannon_list,(attr+'_SIMPSON'):simpson_list})
    return new_df
    

In [27]:
div_df= pd.DataFrame({'idr':comb_df['idr'].unique()})
for col in ['SEX','RELTYPE','ALTERTYPE','AGE35','AGE65','AGE_Q','AGE_10','CESD_16','CESD_20','LONELY_3','SNI_2','EDU_COLLEGE','APOE4']:
    temp = create_diversity_index(comb_df,col)
    div_df = div_df.merge(temp,on='idr',how='left')

/var/folders/v1/zs65yk6n05bc_3ty0jbr0ljsc7z4rq/T/ipykernel_2918/3913421647.py:24: RuntimeWarning: divide by zero encountered in log
  temp = float(p*(np.log(p)))
/var/folders/v1/zs65yk6n05bc_3ty0jbr0ljsc7z4rq/T/ipykernel_2918/3913421647.py:24: RuntimeWarning: invalid value encountered in double_scalars
  temp = float(p*(np.log(p)))
/var/folders/v1/zs65yk6n05bc_3ty0jbr0ljsc7z4rq/T/ipykernel_2918/3913421647.py:23: RuntimeWarning: invalid value encountered in long_scalars
  p = float(div_dict[cat]/sum(div_dict.values()))
/var/folders/v1/zs65yk6n05bc_3ty0jbr0ljsc7z4rq/T/ipykernel_2918/3913421647.py:24: RuntimeWarning: invalid value encountered in log
  temp = float(p*(np.log(p)))
/var/folders/v1/zs65yk6n05bc_3ty0jbr0ljsc7z4rq/T/ipykernel_2918/3913421647.py:35: RuntimeWarning: invalid value encountered in long_scalars
  p = float(div_dict[cat]/sum(div_dict.values()))


In [28]:
## TO DO: check if these egos really dont have alters
div_df.isna().sum()

idr                      0
SEX_SHANNON            140
SEX_SIMPSON            140
RELTYPE_SHANNON        140
RELTYPE_SIMPSON        140
ALTERTYPE_SHANNON      140
ALTERTYPE_SIMPSON      140
AGE35_SHANNON          140
AGE35_SIMPSON          140
AGE65_SHANNON          140
AGE65_SIMPSON          140
AGE_Q_SHANNON            0
AGE_Q_SIMPSON          140
AGE_10_SHANNON           0
AGE_10_SIMPSON         140
CESD_16_SHANNON        623
CESD_16_SIMPSON        623
CESD_20_SHANNON        623
CESD_20_SIMPSON        623
LONELY_3_SHANNON       558
LONELY_3_SIMPSON       558
SNI_2_SHANNON          600
SNI_2_SIMPSON          600
EDU_COLLEGE_SHANNON    722
EDU_COLLEGE_SIMPSON    722
APOE4_SHANNON          369
APOE4_SIMPSON          369
dtype: int64

In [29]:
div_df.shape

(4941, 27)

In [30]:
div_df.describe()

,SEX_SHANNON,SEX_SIMPSON,RELTYPE_SHANNON,RELTYPE_SIMPSON,ALTERTYPE_SHANNON,ALTERTYPE_SIMPSON,AGE35_SHANNON,AGE35_SIMPSON,AGE65_SHANNON,AGE65_SIMPSON,AGE_Q_SHANNON,AGE_Q_SIMPSON,AGE_10_SHANNON,AGE_10_SIMPSON,CESD_16_SHANNON,CESD_16_SIMPSON,CESD_20_SHANNON,CESD_20_SIMPSON,LONELY_3_SHANNON,LONELY_3_SIMPSON,SNI_2_SHANNON,SNI_2_SIMPSON,EDU_COLLEGE_SHANNON,EDU_COLLEGE_SIMPSON,APOE4_SHANNON,APOE4_SIMPSON
count,4801.000000,4801.000000,4801.000000,4801.000000,4801.000000,4801.000000,4801.000000,4801.000000,4801.000000,4801.000000,4941.000000,4801.000000,4941.000000,4801.000000,4318.000000,4318.000000,4318.000000,4318.000000,4383.000000,4383.000000,4341.000000,4341.000000,4219.000000,4219.000000,4572.000000,4572.000000
mean,0.422487,0.296717,0.328289,0.224439,0.703814,0.403700,0.319497,0.221440,0.002045,0.001238,0.446544,0.293974,0.621918,0.381678,0.323271,0.224715,0.187896,0.125487,0.087039,0.055301,0.000160,0.000115,0.301698,0.210577,0.260492,0.175630
std,0.308390,0.219160,0.301708,0.212156,0.532830,0.282682,0.313065,0.220235,0.030141,0.019715,0.410878,0.254229,0.515921,0.285662,0.315345,0.222055,0.267801,0.183638,0.189816,0.126090,0.010520,0.007589,0.318263,0.224398,0.290089,0.200756
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.636514,0.444444,0.410116,0.244898,0.693147,0.500000,0.410116,0.244898,0.000000,0.000000,0.562335,0.375000,0.673012,0.489796,0.429323,0.260355,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.682908,0.489796,0.636514,0.444444,1.098612,0.640000,0.636514,0.444444,0.000000,0.000000,0.693147,0.500000,1.054920,0.627219,0.636514,0.444444,0.450561,0.277778,0.000000,0.000000,0.000000,0.000000,0.636514,0.444444,0.562335,0.375000
max,0.693147,0.500000,0.693147,0.500000,2.106577,0.867347,0.693147,0.500000,0.693147,0.500000,1.386294,0.750000,1.735126,0.816327,0.693147,0.500000,0.693147,0.500000,0.693147,0.500000,0.693147,0.500000,0.693147,0.500000,0.693147,0.500000


In [65]:
export_df

,idr,sharealterid,RELTYPE,ALTERTYPE,CAUSEINIT,CAUSESEVERED,DISTMI1,DISTMI2,DISTMI3,DISTMI4,DISTMI5,DISTMI6,DISTMI7,DISTMI8,EGO_TREIMAN1,EGO_TREIMAN2,EGO_TREIMAN3,EGO_TREIMAN4,EGO_TREIMAN5,EGO_TREIMAN6,EGO_TREIMAN7,EGO_TREIMAN8,ALTER_TREIMAN1,ALTER_TREIMAN2,ALTER_TREIMAN3,ALTER_TREIMAN4,ALTER_TREIMAN5,ALTER_TREIMAN6,ALTER_TREIMAN7,ALTER_TREIMAN8,SPELLBEGIN,SPELLEND,idtype,alteridtype,SEX,AGE1,AGE2,AGE3,AGE4,AGE5,AGE6,AGE7,AGE8,AGE9,G587,G588,G589,G590,G591,G592,G593,G594,G595,G596,G597,G598,G599,G600,G601,G602,G603,G604,G605,G606,G608,G609,G610,G611,G612,G613,G614,G615,G616,G617,G618,G619,G620,G621,G622,G623,G624,G626,G627,G629,G630,G632,G633,G635,G636,G638,G639,G641,G642,G644,G645,G646,G647,G648,G649,G650,G651,G652,G653,G654,G655,G656,G657,EDUCG,EDU_COLLEGE,APOE,APOE4,AGE35,AGE65,AGE_Q,AGE_10,CESD_TOT,CESD_16,CESD_20,CESD_LONELINESS,LONELY_3,SNI_TOT,SNI_2
0,2450200061,2450229337,NONRELATIVE,COWORKERNR,NAMED,NOT NAMED,3.026484,3.026484,3.026484,2.131549,2.131549,2.131549,2.131549,2.131549,40.0,40.0,40.0,31.0,31.0,40.0,40.0,40.0,40.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN,40.0,191.0,1.0,1.0,1.0,30.0,37.0,42.0,45.0,49.0,54.0,56.0,63.0,68.0,1.0,0.0,0.0,3.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,888.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,1.0,NaN,1.0,2.0,1.0,2.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,3.0,0.0,1.0,4.0,4.0,4.0,4.0,4.0,NaN,NaN,34.0,+,<35,<65,"(21.25, 37.5]",<=30,16.0,>=16,<20,0.0,<3,29.0,>2
1,2450200061,2450242028,NONRELATIVE,COWORKERNR,NAMED,NOT NAMED,3.076534,3.076534,3.076534,3.076534,3.076534,3.076534,3.076534,3.076534,40.0,40.0,40.0,31.0,31.0,40.0,40.0,40.0,29.0,29.0,29.0,29.0,29.0,29.0,NaN,NaN,40.0,227.0,1.0,1.0,1.0,30.0,37.0,42.0,45.0,49.0,54.0,56.0,63.0,68.0,1.0,0.0,0.0,3.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,888.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,1.0,NaN,1.0,2.0,1.0,2.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,3.0,0.0,1.0,4.0,4.0,4.0,4.0,4.0,NaN,NaN,34.0,+,<35,<65,"(21.25, 37.5]",<=30,16.0,>=16,<20,0.0,<3,29.0,>2
2,2450200061,2450253095,NONRELATIVE,COWORKERNR,NAMED,NOT NAMED,4.440569,1.204725,1.204725,1.204725,1.204725,1.204725,187.791915,187.791915,40.0,40.0,40.0,31.0,31.0,40.0,40.0,40.0,40.0,40.0,40.0,NaN,NaN,NaN,NaN,NaN,40.0,227.0,1.0,1.0,1.0,30.0,37.0,42.0,45.0,49.0,54.0,56.0,63.0,68.0,1.0,0.0,0.0,3.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,888.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,1.0,NaN,1.0,2.0,1.0,2.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,3.0,0.0,1.0,4.0,4.0,4.0,4.0,4.0,NaN,NaN,34.0,+,<35,<65,"(21.25, 37.5]",<=30,16.0,>=16,<20,0.0,<3,29.0,>2
3,2450200061,2450257865,NONRELATIVE,COWORKERNR,NAMED,NOT NAMED,3.817789,3.817789,3.817789,3.817789,3.817789,75.149427,75.149427,NaN,40.0,40.0,40.0,31.0,31.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,NaN,NaN,NaN,NaN,34.0,221.0,1.0,1.0,1.0,30.0,37.0,42.0,45.0,49.0,54.0,56.0,63.0,68.0,1.0,0.0,0.0,3.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,888.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,1.0,NaN,1.0,2.0,1.0,2.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,3.0,0.0,1.0,4.0,4.0,4.0,4.0,4.0,NaN,NaN,34.0,+,<35,<65,"(21.25, 37.5]",<=30,16.0,>=16,<20,0.0,<3,29.0,>2
4,2450200061,2450261435,NONRELATIVE,COWORKERNR,NAMED,NOT NAMED,3.076534,3.076534,3.076534,3.076534,3.076534,3.076534,3.076534,3.076534,40.0,40.0,40.0,31.0,31.0,40.0,40.0,40.0,53.0,53.0,53.0,53.0,53.0,53.0,NaN,NaN,25.0,215.0,1.0,1.0,1.0,30.0,37.0,42.0,45.0,49.0,54.0,56.0,63.0,68.0,1.0,0.0,0.0,3.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,888.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,1.0,NaN,1.0,2.0,1.0,2.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0,3.0,0.0,1.0,4.0,4.0,4.0,4.0,4.0,NaN,NaN,34.0,+,<35,<65,"(21.25, 37.5]",<=30,16.0,>=16,<20,0.0,<3,29.0,>2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [31]:
export_df = comb_df[['idr',
                     'sharealterid',
                     'RELTYPE',
                     'ALTERTYPE',
                     'SEX',
                     'AGE1',
                     'idtype',
                     'alteridtype',
                     'SPELLBEGIN',
                     'SPELLEND',
                     'APOE',
                     'EDUCG',
                     'CESD_TOT',
                     'CESD_LONELINESS',
                     'SNI_TOT'
                    ]].merge(div_df,on='idr')

In [34]:
export_df.groupby('idr',as_index=False).first()

,idr,sharealterid,RELTYPE,ALTERTYPE,SEX,AGE1,idtype,alteridtype,SPELLBEGIN,SPELLEND,APOE,EDUCG,CESD_TOT,CESD_LONELINESS,SNI_TOT,SEX_SHANNON,SEX_SIMPSON,RELTYPE_SHANNON,RELTYPE_SIMPSON,ALTERTYPE_SHANNON,ALTERTYPE_SIMPSON,AGE35_SHANNON,AGE35_SIMPSON,AGE65_SHANNON,AGE65_SIMPSON,AGE_Q_SHANNON,AGE_Q_SIMPSON,AGE_10_SHANNON,AGE_10_SIMPSON,CESD_16_SHANNON,CESD_16_SIMPSON,CESD_20_SHANNON,CESD_20_SIMPSON,LONELY_3_SHANNON,LONELY_3_SIMPSON,SNI_2_SHANNON,SNI_2_SIMPSON,EDU_COLLEGE_SHANNON,EDU_COLLEGE_SIMPSON,APOE4_SHANNON,APOE4_SIMPSON
0,2450200061,2450229337,NONRELATIVE,COWORKERNR,1.0,30.0,1.0,1.0,40.0,191.0,34.0,NaN,16.0,0.0,29.0,0.650818,0.458272,0.348832,0.197531,0.794939,0.378272,0.601154,0.410864,0.0,0.0,0.780264,0.517531,1.213494,0.650864,0.622487,0.431020,0.410116,0.244898,0.336496,0.188366,0.0,0.0,0.684232,0.491111,0.623655,0.432133
1,2450200238,2450320467,RELATIVE,FATHER,2.0,16.0,1.0,0.0,1.0,192.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.500000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
2,2450200299,2450340036,NONRELATIVE,SAMEADNREL,1.0,26.0,1.0,1.0,35.0,137.0,23.0,NaN,NaN,NaN,NaN,0.000000,0.000000,0.693147,0.500000,0.693147,0.500000,0.000000,0.000000,0.0,0.0,0.693147,0.500000,0.693147,0.500000,0.693147,0.500000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.693147,0.500000
3,2450200493,2450203175,RELATIVE,BROTHER,2.0,26.0,1.0,1.0,1.0,457.0,NaN,NaN,NaN,NaN,NaN,0.673012,0.480000,0.000000,0.000000,0.500402,0.320000,0.000000,0.000000,0.0,0.0,0.500402,0.320000,1.054920,0.640000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.693147,0.500000
4,2450200742,2450370277,RELATIVE,SISTER,1.0,27.0,1.0,1.0,1.0,457.0,34.0,3.0,12.0,0.0,27.0,0.500402,0.320000,0.673012,0.480000,1.054920,0.640000,0.673012,0.480000,0.0,0.0,0.500402,0.320000,0.950271,0.560000,0.693147,0.500000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.562335,0.375000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4936,2450949062,2450226957,RELATIVE,MOTHER,2.0,27.0,1.0,0.0,1.0,80.0,34.0,NaN,8.0,0.0,32.0,0.673012,0.480000,0.000000,0.000000,1.054920,0.640000,0.000000,0.000000,0.0,0.0,0.500402,0.320000,0.950271,0.560000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.562335,0.375000,0.500402,0.320000
4937,2450949332,2450252749,RELATIVE,MOTHER,2.0,30.0,1.0,0.0,1.0,416.0,33.0,3.0,12.0,0.0,27.0,0.693147,0.500000,0.562335,0.375000,1.039721,0.625000,0.562335,0.375000,0.0,0.0,0.562335,0.375000,0.000000,0.000000,0.562335,0.375000,0.562335,0.375000,0.562335,0.375000,0.0,0.0,0.562335,0.375000,0.693147,0.500000
4938,2450949473,2450233240,NONRELATIVE,N100MNREL,2.0,51.0,1.0,1.0,28.0,129.0,33.0,1.0,14.0,0.0,20.0,0.661563,0.468750,0.000000,0.000000,0.562335,0.375000,0.562335,0.375000,0.0,0.0,1.073543,0.562500,1.039721,0.625000,0.693147,0.500000,0.636514,0.444444,0.450561,0.277778,0.0,0.0,0.693147,0.500000,0.376770,0.218750
4939,2450949485,2450470004,RELATIVE,SPOUSE,2.0,52.0,1.0,1.0,1.0,304.0,33.0,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000


In [35]:
#to share
export_df.to_csv('fhs_sn_diversity.csv')